In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/combined_trajectories.csv')
df.head()

In [ ]:
df.info()

In [ ]:
print(len(df['individual_id'].unique()))

182


In [ ]:
df2= df[['latitude', 'longitude', 'altitude']]
point = [list(x) for x in df2.to_numpy()]


In [ ]:
print(len(point))

24876978


In [ ]:
print(point[1])

[39.984683, 116.31845, 492.0]


In [ ]:
df['coordinate'] = point

In [ ]:
df.head()

,latitude,longitude,altitude,date,time,individual_id,trajectory_id,coordinate
0,39.984702,116.318417,492.0,2023-10-23,02:53:04,1,1,"[39.984702, 116.318417, 492.0]"
1,39.984683,116.318450,492.0,2023-10-23,02:53:10,1,1,"[39.984683, 116.31845, 492.0]"
2,39.984686,116.318417,492.0,2023-10-23,02:53:15,1,1,"[39.984686, 116.318417, 492.0]"
3,39.984688,116.318385,492.0,2023-10-23,02:53:20,1,1,"[39.984688, 116.318385, 492.0]"
4,39.984655,116.318263,492.0,2023-10-23,02:53:25,1,1,"[39.984655, 116.318263, 492.0]"


In [ ]:
import concurrent.futures
from math import radians, cos, sin, sqrt, atan2

In [ ]:
def calculate_distance(p1, p2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1 = map(radians, p1[:2])
    lat2, lon2 = map(radians, p2[:2])

    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = 6371 * c * 1000  # Radius of Earth in meters

    alt1, alt2 = p1[2], p2[2]
    if alt1 is not None and alt2 is not None:
        distance += abs(alt2 - alt1)

    return distance


In [ ]:
df.iloc[1]['coordinate']

[39.984683, 116.31845, 492.0]

In [ ]:


def multithreaded(df, chunk_size=1000):
    total_distance = {}

    def calculate_chunk_distance(chunk):
        chunk_distance = {}
        for individual_id, coordinate in chunk:
            distances = [calculate_distance(df.iloc[i]['coordinate'], df.iloc[i+1]['coordinate']) for i in range(len(coordinate)-1)]
            total_distance_traveled = sum(distances)
            chunk_distance[individual_id] = total_distance_traveled

        return chunk_distance

    chunks = [{individual_id: coordinate} for individual_id,coordinate  in df.items()]
    chunks = [chunks[i:i+chunk_size] for i in range(0, len(chunks), chunk_size)]

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(calculate_chunk_distance, chunk) for chunk in chunks]


        for future in concurrent.futures.as_completed(futures):
            chunk_distance = future.result()
            for individual_id, distance in chunk_distance.items():
                if individual_id in total_distance:
                    total_distance[individual_id] += distance
                else:
                    total_distance[individual_id] = distance

    return total_distance


In [ ]:
df2= df.loc[df['individual_id'] == 1]

In [ ]:
df2 = df.head(1000)

NameError: ignored

In [ ]:
import folium

beijing_map = folium.Map(location=[39.9042, 116.4074], zoom_start=12)


for _, row in df2.iterrows():
    lat = row['latitude']
    lon = row['longitude']
    alt = row['altitude']


    popup_msg = 'Altitude: {} meters'.format(alt) if not pd.isnull(alt) else 'Altitude: unknown'

    # Add a marker for the point to the map
    marker = folium.Marker(location=[lat, lon], popup=popup_msg)
    marker.add_to(beijing_map)

# Display the map
beijing_map


In [ ]:
import pandas as pd
import folium


df = pd.read_csv('path/to/dataset.csv')


beijing_map = folium.Map(location=[39.9042, 116.4074], zoom_start=12)


for user_id in df.groupby('individual_id'):
    # Create a feature group for this user's trajectory
    user_traj = folium.FeatureGroup(name='User {}'.format(user_id))

    # Iterate over the rows of this user's trajectory and add markers for each point
    for _, row in user_group.iterrows():
        # Get the latitude, longitude, and altitude values
        lat = row['latitude']
        lon = row['longitude']
        alt = row['altitude']

        # Create a popup message to display the altitude value
        popup_msg = 'Altitude: {} meters'.format(alt) if not pd.isnull(alt) else 'Altitude: unknown'

        # Add a marker for the point to the user's trajectory
        marker = folium.Marker(location=[lat, lon], popup=popup_msg)
        marker.add_to(user_traj)

    # Add the user's trajectory to the map
    user_traj.add_to(beijing_map)

# Add a layer control to the map
folium.LayerControl().add_to(beijing_map)

# Display the map
beijing_map


In [ ]:
from sklearn.cluster import KMeans

# Select only the latitude and longitude columns of the dataset
X = df[['latitude', 'longitude']]

# Fit a KMeans model with 10 clusters
kmeans = KMeans(n_clusters=10).fit(X)

# Add the cluster centers as markers to the map
cluster_centers = kmeans.cluster_centers_
for center in cluster_centers:
    marker = folium.Marker(location=[center[0], center[1]], icon=folium.Icon(color='red'))
    marker.add_to(beijing_map)

# Display the map
beijing_map
